# Math Grade Predictor

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

### Utility Functions

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Utility function to print MSE, MAE and R2 metrics
def calc_MSE_MAE_R2(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    print(f"MSE: {mse}\nMAE: {mae}\nR2: {r2}")


### Preprocessing

In [4]:
# Reading initial data
df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')

In [5]:
# Matching train columns with test columns
df_train_proc = df_train.drop(columns=df_train.columns[~df_train.columns.isin(df_test.columns)])
df_train_proc = pd.concat([df_train_proc, df_train['NU_NOTA_MT']], axis=1)

# Drop every row where NU_NOTA_MT is null
# df_train_proc.dropna(axis=0, subset=['NU_NOTA_MT'], inplace=True)

# Fill null values of NU_NOTA_MT with 0s
df_train_proc['NU_NOTA_MT'].fillna(value=0, inplace=True)

In [6]:
# Split train and test
y = df_train_proc['NU_NOTA_MT']
X = df_train_proc.drop(columns=['NU_NOTA_MT'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
# Workaround to fix a testset incompleteness
new_row = df_train_proc.iloc[3, :].copy()
new_row.name = 50000
new_row['NU_INSCRICAO', 'CO_PROVA_CH', 'CO_PROVA_CN'] = ['73b4e129e9ffe8ab17f6d094db1876ef3f363e1b', 
                                                         'd5f6d17523d2cce3e4dc0a7f0582a85cec1c15ee', 
                                                         'a27a1efea095c8a973496f0b57a24ac6775d95b0']
new_row = new_row.drop(labels=['NU_NOTA_MT'])
X_test = X_test.append(new_row)
df_test = df_test.append(new_row)
y_test = y_test.append(pd.Series(df_train_proc.iloc[3,-1].copy()))

In [8]:
# Pipeline to fill null values
fill_na_cols = ['TP_SEXO', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 
                'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047']
fill_na_pipeline = Pipeline([('fillna', SimpleImputer(strategy='constant', fill_value='0'))])
onehot_pipeline = Pipeline([('onehot', OneHotEncoder())])

# Pipeline to impute median
median_cols = ['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_STATUS_REDACAO']
median_pipeline = Pipeline([('median', SimpleImputer(strategy='median'))])

# Pipeline to impute mean
mean_cols = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 
             'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']
mean_pipeline = Pipeline([('mean', SimpleImputer(strategy='mean'))])

# Dropping NU_INSCRICAO column because of it's high variance.
# Dropping SG_UF_RESIDENCIA column because CO_UF_RESIDENCIA is a numerical reference to it.
drop_cols = ['NU_INSCRICAO', 'SG_UF_RESIDENCIA']

# Preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('drop', 'drop', drop_cols),
    ('fillna', fill_na_pipeline, fill_na_cols),
    ('median', median_pipeline, median_cols),
    ('mean',  mean_pipeline, mean_cols)
], remainder='passthrough')

# Training pipeline
std_pca_lasso_pipeline = Pipeline([
    ('std', StandardScaler(with_mean=False)),
    ('svd', TruncatedSVD(100)),
    ('pca', PCA()),
    ('rfr', RandomForestRegressor())
])

# Unifying the pipelines into one
pipeline = make_pipeline(
    preprocessor,
    OneHotEncoder(handle_unknown='ignore'),
    std_pca_lasso_pipeline
)

### Training

In [9]:
# Training model
pipe = pipeline.fit(X_train, y_train)
result = pipeline.predict(X_test)

# Showing Metrics
calc_MSE_MAE_R2(y_test, result)

MSE: 5146.257702927184
MAE: 47.114195145631065
R2: 0.9043301038678958


In [10]:
# Show Score
pipeline.score(X_train, y_train)

0.9678228449410695

In [11]:
# Predicting Math Grades
y_final = pipeline.predict(df_test)

### Preparing the output file

In [12]:
# Preparing and creating the answer.csv
answer = pd.DataFrame(df_test['NU_INSCRICAO'])
answer.drop(labels=50000, inplace=True)
answer = pd.concat([answer, pd.Series(y_final)], axis=1, join='inner')
answer.columns = ['NU_INSCRICAO', 'NU_NOTA_MT']
answer.to_csv('answer.csv', index=False)
answer.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,439.366
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,480.940
2,b38a03232f43b11c9d0788abaf060f7366053b6d,497.952
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.000
4,715494628a50142ce8cb17191cfe6d0f3cae0934,443.237
